In [ ]:
!pip install yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, Concatenate, Attention, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras import Model, regularizers

In [ ]:
def preprocess(df):
    # Calculate RSI
    def calc_rsi(close, period):
        deltas = close.diff()
        gains = deltas.clip(lower=0)
        losses = -deltas.clip(upper=0)
        avg_gain = gains.rolling(window=period, min_periods=1).mean()
        avg_loss = losses.rolling(window=period, min_periods=1).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (rs + 1))
        return rsi

    # Calculate OBV
    def calc_obv(df):
        obv = ((df['Close'] > df['Close'].shift(1)) * 2 - 1) * df['Volume']
        return obv.cumsum()

    # Calculate MACD
    def calc_macd(df):
        ema12 = df['Close'].ewm(span=12, adjust=False).mean()
        ema26 = df['Close'].ewm(span=26, adjust=False).mean()
        macd = ema12 - ema26
        macd_signal = macd.ewm(span=9, adjust=False).mean()
        macd_histogram = macd - macd_signal
        return macd, macd_signal, macd_histogram

    # Calculate ATR
    def calculate_atr(df, period=3):
        high_low = df['High'] - df['Low']
        high_previous_close = np.abs(df['High'] - df['Close'].shift(1))
        low_previous_close = np.abs(df['Low'] - df['Close'].shift(1))
        true_range = pd.DataFrame({
            'High-Low': high_low, 
            'High-PreviousClose': high_previous_close, 
            'Low-PreviousClose': low_previous_close
        })
        tr = true_range.max(axis=1)
        atr = tr.rolling(window=period, min_periods=1).mean()
        return atr

    # Add technical indicators
    def add_technicals(df):
        df['MA5'] = df['Close'].rolling(5).mean()
        df['MA10'] = df['Close'].rolling(10).mean()
        df['RSI'] = calc_rsi(df['Close'], period=14)
        df['OBV'] = calc_obv(df)
        df['MACD'], df['MACD Signal'], df['MACD Histogram'] = calc_macd(df)
        df['ATR'] = calculate_atr(df)
        return df

    # Remove 'Adj Close' if present
    df = add_technicals(df)
    df = df.drop(columns=['Adj Close', 'Volume'], errors='ignore')

    # Create Target columns for the next day's prices
    df['Target1'] = df['Close'].shift(-1)
    df['Target2'] = df['Close'].shift(-2)
    df['Target3'] = df['Close'].shift(-3)

    return df.dropna()
df = yf.download('SPY', progress=False)
df = preprocess(df)

In [ ]:
# Scale the features and target
scalers = {}
target_columns = ['Target1', 'Target2', 'Target3']
feature_columns = list(df.columns)
for i in target_columns:
    feature_columns.remove(i)
print(feature_columns)
for col in feature_columns:
    scaler = StandardScaler()
    df[col] = scaler.fit_transform(df[[col]])
    scalers[col] = scaler

# Scale the target column separately
target_scaler = StandardScaler()
df['Target1'] = target_scaler.fit_transform(df[['Target1']])
scalers['Target1'] = target_scaler
target_scaler = StandardScaler()
df['Target2'] = target_scaler.fit_transform(df[['Target2']])
scalers['Target2'] = target_scaler
target_scaler = StandardScaler()
df['Target3'] = target_scaler.fit_transform(df[['Target3']])
scalers['Target3'] = target_scaler

In [ ]:
def sequencer(df, sl):
    xs, ys = [], []
    num_rows = len(df)
    for i in range(0, num_rows - sl + 1):
        chunk = df.iloc[i:i + sl]
        nchunk = df.iloc[i+1:i+sl+1]
        features = chunk.drop(target_columns, axis=1).values
        target = nchunk[target_columns].values[0]
        xs.append(features)
        ys.append(target)
    return np.array(xs), np.array(ys)

# Create sequences
sequence_length = 15
X_sequences, y_sequences = sequencer(df, sequence_length)

print(f'X_sequences shape: {X_sequences.shape}')
print(f'y_sequences shape: {y_sequences.shape}')

In [ ]:
# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, Concatenate, Attention, Conv1D, MaxPooling1D, Flatten, TimeDistributed
from tensorflow.keras import Model, regularizers

def build_model(sequence_length=15, num_features=12):
    inputs = Input(shape=(sequence_length, num_features))

    conv = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(inputs)
    pool = MaxPooling1D(pool_size=2)(conv)
    flat = Flatten()(pool)

    lstm_out = LSTM(32, return_sequences=True)(inputs)

    gru_out = GRU(32, return_sequences=True)(inputs)

    attention = Attention()([lstm_out, gru_out])

    attention_flat = Flatten()(attention)

    dense_lstm = Dense(16, activation='relu')(lstm_out)
    dense_gru = Dense(16, activation='relu')(gru_out)

    dense_lstm_flat = Flatten()(dense_lstm)
    dense_gru_flat = Flatten()(dense_gru)

    concat = Concatenate()([flat, attention_flat, dense_lstm_flat, dense_gru_flat])

    outputs = Dense(3, activation='linear')(concat)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss='mse', metrics=['mape'])
    return model

In [ ]:
model = build_model()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(
    model,
    to_file='model.png',
    dpi=2400,
    show_shapes=True,
    show_layer_activations=True)

In [ ]:
mcb = [
    tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=100, min_delta=1e-5),
]
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=500,
    batch_size=8,
    callbacks=mcb,
)

In [ ]:
# Plot training & validation loss
plt.plot(history.history['val_loss'], color='green', label='val_loss')
plt.plot(history.history['loss'], color='blue', label='loss')
plt.grid()
plt.legend()
plt.show()

# Plot training & validation MAPE
plt.plot(history.history['val_mape'], color='green', label='val_mape')
plt.plot(history.history['mape'], color='blue', label='mape')
plt.grid()
plt.legend()
plt.show()

In [ ]:
import keras
model = keras.saving.load_model('best_model.keras')

In [ ]:
model.evaluate(X_test, y_test)